In [50]:
from hyperopt import hp
import numpy as np

from recpack.algorithms import ItemKNN
from recpack.datasets import DummyDataset
from recpack.pipelines import PipelineBuilder, HyperoptInfo
from recpack.scenarios import WeakGeneralization

In [51]:
HOUR = 3600
DAY = 24*3600

optimisation_info = HyperoptInfo(
    hp.choice(
        'similarity', [
            {
                'similarity': 'conditional_probability', 
                'pop_discount': hp.uniform('pop_discount', 0, 1),
                "K": hp.loguniform("cp_K", np.log(50), np.log(1000)),
                "normalize_X": hp.choice("cp_normalize_X", [True, False]),
                "normalize_sim": hp.choice("cp_normalize_sim",[True, False]),
            }, 
            {
                'similarity': 'cosine',
                "K": hp.loguniform("c_K", np.log(50), np.log(1000)),
                "normalize_X": hp.choice("c_normalize_X", [True, False]),
                "normalize_sim": hp.choice("c_normalize_sim",[True, False]),
            }
        ]
    ),
    timeout=DAY,
    max_evals=50,
)


In [52]:
im = DummyDataset().load()
scenario = WeakGeneralization(validation=True)
scenario.split(im)

100%|██████████| 490/490 [00:00<00:00, 422447.88it/s]
92it [00:00, 2505.53it/s]
92it [00:00, 2786.95it/s]


In [ ]:
pb = PipelineBuilder()
pb.add_algorithm(ItemKNN, optimisation_info=optimisation_info)
pb.set_data_from_scenario(scenario)
pb.set_optimisation_metric('NDCGK', 10)
pb.add_metric('NDCGK', 10)

pipe = pb.build()
pipe.run()


In [54]:
pipe.get_metrics()

,ndcgk_10
"ItemKNN(K=602.3509511335283,normalize_X=True,normalize_sim=False,pop_discount=None,similarity=cosine)",0.058466
